In [ ]:
import os
import cv2 
import numpy as np
import keras 


In [2]:
img=[]

In [3]:
labels=[]

# taking data from pc

In [4]:
#the location of the data was as below when i took into my laptop

In [5]:
os.listdir("/home/swamikedari/Arificial Intelligence/dataset-20220716T053416Z-001/dataset")

['0', '1']

In [6]:
# i was getting errors in due to the file name so i changed it to 0 and 1 in my laptop

In [7]:
for i in [0 ,1]:
    imglist=os.listdir("/home/swamikedari/Arificial Intelligence/dataset-20220716T053416Z-001/dataset/"+str(i))
    for filename in imglist:
        image=cv2.imread("/home/swamikedari/Arificial Intelligence/dataset-20220716T053416Z-001/dataset/"+str(i)+"/"+str(filename))
        try:
            a=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
            image=cv2.resize(a,(100,100))
            img.append(image)
            labels.append(i)
        except:
            pass
        

In [8]:
img=np.array(img)

In [9]:
img.shape

(1376, 100, 100)

In [10]:
def div(x):
    x=x/255
    return x

In [11]:
t=list(map(div,image))
image=np.array(t)

In [12]:
labels=np.array(labels)

# splitting the data into training and testing features and targets

In [13]:
from sklearn.model_selection import train_test_split


In [14]:
a,b,c,d=train_test_split(img,labels,test_size=0.2)

In [15]:
a.shape

(1100, 100, 100)

In [16]:
a=a.reshape(a.shape[0],a.shape[1],a.shape[2],1)

In [17]:
a.shape

(1100, 100, 100, 1)

In [18]:
b=b.reshape(b.shape[0],b.shape[1],b.shape[2],1)

In [19]:
b.shape

(276, 100, 100, 1)

# Image augmentation

In [20]:
from keras.preprocessing.image import ImageDataGenerator

In [21]:
dg=ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2,shear_range=0.1,rotation_range=10)

In [22]:
dg.fit(a)

In [23]:
batches=dg.flow(a,c,batch_size=20)

In [24]:
a.shape

(1100, 100, 100, 1)

In [25]:
c.shape

(1100,)

# increasing the number of targets

In [26]:
from keras.utils import to_categorical

In [27]:
c=to_categorical(c)

In [28]:
c.shape

(1100, 2)

In [29]:
from keras.models import Sequential
from keras.layers import MaxPooling2D,Dense,Conv2D,Flatten,Dropout



# specify the architecture

In [30]:
model=Sequential()
model.add(Conv2D(128,(3,3),activation="relu",input_shape=(100,100,1)))
model.add(Conv2D(128,(3,3),activation="relu"))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(Conv2D(100,(3,3),activation="relu"))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(750,activation="relu"))
model.add(Dense(350,activation="relu"))
model.add(Dense(350,activation="relu"))
model.add(Dense(2,activation="softmax"))

2022-07-16 15:27:14.105292: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-16 15:27:14.105318: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2022-07-16 15:27:14.105626: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# compile

In [31]:
from keras.optimizers import Adam

In [32]:
model.compile(Adam(lr=0.0001),loss="categorical_crossentropy",metrics=["accuracy"])

/home/swamikedari/.local/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


# fit

In [33]:
model.fit_generator(dg.flow(a,c,batch_size=5),epochs=20)

Epoch 1/20


/tmp/ipykernel_59482/4024386068.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(dg.flow(a,c,batch_size=5),epochs=20)


220/220 [==============================] - 47s 209ms/step - loss: 0.7865 - accuracy: 0.5755
Epoch 2/20
220/220 [==============================] - 49s 224ms/step - loss: 0.5838 - accuracy: 0.6900
Epoch 3/20
220/220 [==============================] - 49s 225ms/step - loss: 0.4231 - accuracy: 0.8055
Epoch 4/20
220/220 [==============================] - 50s 226ms/step - loss: 0.3071 - accuracy: 0.8745
Epoch 5/20
220/220 [==============================] - 50s 226ms/step - loss: 0.3150 - accuracy: 0.8782
Epoch 6/20
220/220 [==============================] - 50s 226ms/step - loss: 0.2350 - accuracy: 0.9127
Epoch 7/20
220/220 [==============================] - 50s 227ms/step - loss: 0.1907 - accuracy: 0.9200
Epoch 8/20
220/220 [==============================] - 51s 230ms/step - loss: 0.1488 - accuracy: 0.9455
Epoch 9/20
220/220 [==============================] - 50s 227ms/step - loss: 0.1401 - accuracy: 0.9500
Epoch 10/20
220/220 [==============================] - 50s 229ms/step - loss: 0.1536

In [34]:
a.shape

(1100, 100, 100, 1)

# taking pictures from the camera

In [49]:
def classtype(x):
    if x==0: return "with      mask"
    elif x==1 : return "with     out       mask"

In [56]:
cp=cv2.VideoCapture(0)
cp.set(3,640)
cp.set(4,480)
cp.set(10,100)

True

In [51]:
face_cascade=cv2.CascadeClassifier("/home/swamikedari/Arificial Intelligence/haarcascade_frontalface_default.xml")

In [57]:
font=cv2.FONT_HERSHEY_SIMPLEX
while True:
    success,imgcam=cp.read()
    j=np.array(imgcam)
    j=cv2.resize(j,(100,100))
    j=cv2.cvtColor(j,cv2.COLOR_BGR2GRAY)
    face=face_cascade.detectMultiScale(j,1.1,1)
    for(x,y,w,h) in face:
        cv2.rectangle(j,(x,y),(x+w,y+h),(255,0,0),3)
        j=face[x:x+w,y:y+h]
    j=div(j)
    #cv2.imshow("processed",j)
    j=j.reshape(1,100,100,1)
    cv2.putText(imgcam,("class  : "),(20,35),font,1,(0,0,255),2,cv2.LINE_AA)
    cv2.putText(imgcam,("status  :"),(20,80),font,1,(0,0,255),2,cv2.LINE_AA)
    pred=model.predict(j)
    cl=np.argmax(pred,axis=1)
    probabilityValue=np.amax(pred)
    if probabilityValue>0.9:
        cv2.putText(imgcam,str(cl),(200,35),font,1,(0,0,255),2,cv2.LINE_AA)
        cv2.putText(imgcam,str(classtype(cl)),(200,80),font,1,(0,0,255),2,cv2.LINE_AA)
    cv2.imshow("result",imgcam)
    if cv2.waitKey(1)==113:
        cv2.destroyAllWindows()
        cp.release()
        break
    

1/1 [==============================] - 0s 31ms/step


In [38]:
probabilityValue

0.65228164